# Assignment 3 - Extra : CIFAR-10

https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

## Import Data

In [ ]:
! nvidia-smi

Sat Aug  1 08:13:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=bc5b0b3d2ccafa91862e75f1cb53d1d5f9716c0c31340ad41cf83dbfa408997e
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [ ]:
import numpy as np
import pickle
import tensorflow as tf

time: 1.21 s


In [ ]:
tf.__version__

'2.2.0'

time: 2.7 ms


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 13s 0us/step
time: 16.3 s


In [ ]:
X_train.shape

(50000, 32, 32, 3)

time: 5.67 ms


## Preprocessing

In [ ]:
import tensorflow as tf

ImgDatagenProperty = {
    'rotation_range' : 40,
    'width_shift_range' : 0.4,
    'height_shift_range' : 0.4,
    'shear_range' : 0.3,
    'zoom_range' : 0.3,
    'horizontal_flip' : True,
    'fill_mode' : 'nearest',
    'preprocessing_function' : tf.keras.applications.densenet.preprocess_input,
}

time: 4.14 ms


In [ ]:
ImgValDatagenProperty = {
    'preprocessing_function' : tf.keras.applications.densenet.preprocess_input,
}

time: 995 µs


In [ ]:
ImgModelProperty = {
    'img_height' : 32,
    'img_width' : 32,
    'num_class' : 10,
    'epoch' : 60,
    'batch_size' : 128
}

time: 1.81 ms


In [ ]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**ImgDatagenProperty).flow(X_train, y_train, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**ImgValDatagenProperty).flow(X_test, y_test, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)

time: 215 ms


## Model

https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet101

In [ ]:
import tensorflow as tf

def BuildModel(input_dim = (32, 32, 3), n_class = 10) -> tf.keras.Sequential :
    model = tf.keras.Sequential()
    model.add(tf.keras.applications.DenseNet169(include_top=False, weights='imagenet', input_shape=input_dim))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(128, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.BatchNormalization())
    # model.add(tf.keras.layers.Dense(128, activation=tf.keras.activations.relu))
    # model.add(tf.keras.layers.BatchNormalization())
    # model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))
    # model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax))
    # padding_input = tf.keras.layers.ZeroPadding2D(padding=(96, 96), input_shape=input_dim)
    # pretrained = tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')(padding_input)

    # model = tf.keras.Model(padding_input.input, pretrained)
    return model    

time: 11.5 ms


In [ ]:
model = BuildModel()

time: 13.6 s


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 1, 1, 1664)        12642880  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1664)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               213120    
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 12,857,802
Trainable params: 12,699,146
Non-trainable params: 158,656
_________________________________________________________________
time: 44.5 ms


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

time: 29.8 ms


In [ ]:
import tensorflow as tf

red = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='auto', min_lr=1e-7)

time: 1.65 ms


In [ ]:
%load_ext tensorboard
! rm -rf ./logs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
time: 3.38 s


In [ ]:
import tensorflow as tf

esb = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

time: 1.66 ms


In [ ]:
import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tsb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

time: 2.92 ms


In [ ]:
model.fit(
    train_datagen, 
    epochs=ImgModelProperty['epoch'],
    steps_per_epoch=len(X_train) / ImgModelProperty['batch_size'],
    validation_data=val_datagen,
    callbacks=[red, tsb],
)

Epoch 1/60
391/390 [==============================] - 55s 141ms/step - loss: 1.4576 - sparse_categorical_accuracy: 0.4840 - val_loss: 1.4291 - val_sparse_categorical_accuracy: 0.5667 - lr: 0.0010
Epoch 2/60
391/390 [==============================] - 51s 131ms/step - loss: 1.2227 - sparse_categorical_accuracy: 0.5716 - val_loss: 5.5896 - val_sparse_categorical_accuracy: 0.5135 - lr: 0.0010
Epoch 3/60
391/390 [==============================] - 51s 130ms/step - loss: 1.1545 - sparse_categorical_accuracy: 0.5951 - val_loss: 1.0235 - val_sparse_categorical_accuracy: 0.6643 - lr: 0.0010
Epoch 4/60
391/390 [==============================] - 51s 131ms/step - loss: 1.0420 - sparse_categorical_accuracy: 0.6344 - val_loss: 3.6531 - val_sparse_categorical_accuracy: 0.6277 - lr: 0.0010
Epoch 5/60
391/390 [==============================] - 52s 133ms/step - loss: 1.1081 - sparse_categorical_accuracy: 0.6131 - val_loss: 0.9542 - val_sparse_categorical_accuracy: 0.6704 - lr: 0.0010
Epoch 6/60
391/390 [

time: 51min 4s


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3)).summary()

time: 1.16 ms
